# End-to-End TensorFlow Pipeline Notebook

Dieses Notebook führt dich Schritt für Schritt durch einen kompletten Machine-Learning-Workflow mit TensorFlow:

1. **Daten-Pipeline**: Laden, Vorverarbeiten und Batchen von Bildern mit `tf.data`.
2. **TFRecord-Erstellung**: Konvertiere deine Bilder und Labels in das effiziente TFRecord-Format.
3. **Basic CNN**: Implementiere ein einfaches Convolutional Neural Network (CNN) von Grund auf.
4. **Pretrained ResNet**: Nutze ein ResNet-Modell mit vortrainierten Gewichten.
5. **Modellspeicherung**: Speichere und lade dein trainiertes Modell.
6. **Evaluation**: Beurteile die Modellleistung auf einem Evaluierungsdatensatz.

Jede Zelle enthält ausführliche Erklärungen und Verweise auf die TensorFlow-Dokumentation.

## 1. Setup und Imports

Wir starten mit den benötigten Bibliotheken. Stelle sicher, dass TensorFlow 2.x installiert ist.

Referenzen:
- TensorFlow Installationsanleitung: https://www.tensorflow.org/install
- TensorFlow 2 Guide: https://www.tensorflow.org/guide

In [ ]:
import tensorflow as tf
import numpy as np
import os
from pathlib import Path

## 2. Daten-Pipeline mit `tf.data`

Wir erstellen eine Pipeline, die Bilder aus einem Verzeichnis lädt, skaliert und in Batches organisiert.

- **`tf.data.Dataset.list_files`**: Findet Bilddateien
- **`tf.data.Dataset.map`**: Wendet Vorverarbeitung an
- **`tf.data.Dataset.batch`** und **`prefetch`**: Effizientes Batch-Training

Referenz:
- `tf.data` API: https://www.tensorflow.org/guide/data

In [ ]:
# Pfad zu deinem Bild-Ordner (Unterordner je Klasse)
data_dir = Path("./images")

# Parameter
auto = tf.data.AUTOTUNE
batch_size = 4
img_size = (256, 256)

# Funktion zur Verarbeitung jedes Bildes und Labels
def process_path(file_path):
    label = tf.strings.split(file_path, os.path.sep)[-2]
    label = tf.cast(label == class_names, tf.int32)
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, img_size)
    image = image / 255.0  # Normalisierung
    return image, label

# Klassen extrahieren
class_names = np.array(sorted([d.name for d in data_dir.iterdir() if d.is_dir()]))

# Dataset erstellen
autotune = tf.data.AUTOTUNE
list_ds = tf.data.Dataset.list_files(str(data_dir / "*" / "*.png"), shuffle=True)
processed_ds = list_ds.map(process_path, num_parallel_calls=autotune)
dataset = processed_ds.batch(batch_size).prefetch(autotune)

print(dataset)

## 3. TFRecord-Datensatz erstellen

Wir speichern die vorverarbeiteten Bilder und Labels im TFRecord-Format.

- **`tf.train.Example`**: Struktur für einzelne Instanzen
- **`tf.io.TFRecordWriter`**: Schreibt binäre Dateien

Referenz:
- TFRecord Guide: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [ ]:
# Feature-Helfer
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Pfad für TFRecord-File
record_file = './data/dataset.tfrecord'

with tf.io.TFRecordWriter(record_file) as writer:
    for images, labels in dataset:
        for img, lbl in zip(images, labels):
            img_raw = tf.io.encode_jpeg(tf.cast(img*255, tf.uint8)).numpy()
            feature = {
                'image_raw': _bytes_feature(img_raw),
                'label': _int64_feature(int(lbl.numpy().argmax()))
            }
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
print(f"TFRecord erstellt: {record_file}")

## 4. TFRecord-Dataset laden

Wir laden den TFRecord-Datensatz und wandeln die `Example`-Protobufs zurück in Tensors um.

In [ ]:
raw_dataset = tf.data.TFRecordDataset(record_file)

# Schema definieren
def parse_example(example_proto):
    feature_description = {
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64)
    }
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.io.decode_jpeg(parsed['image_raw'], channels=3)
    image = tf.image.resize(image, img_size) / 255.0
    label = tf.one_hot(parsed['label'], depth=len(class_names))
    return image, label

parsed_ds = raw_dataset.map(parse_example, num_parallel_calls=autotune)
dataset = parsed_ds.batch(batch_size).prefetch(autotune)

print(dataset)

## 5. Basic CNN erstellen und trainieren

Ein einfaches CNN zum Einstieg:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(*img_size, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Training
epochs = 10
history = model.fit(dataset, epochs=epochs, validation_data=dataset)

## 6. Pretrained ResNet nutzen

Wir verwenden `tf.keras.applications.ResNet50` mit vortrainierten ImageNet-Gewichten.

Referenz:
- Keras Applications: https://www.tensorflow.org/api_docs/python/tf/keras/applications

In [ ]:
base_model = tf.keras.applications.ResNet50(
    input_shape=(*img_size, 3), include_top=False, weights='imagenet'
)
base_model.trainable = False  # Gefrieren der Basisschichten

inputs = tf.keras.Input(shape=(*img_size, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)
resnet_model = tf.keras.Model(inputs, outputs)

resnet_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

resnet_model.summary()

# Feinabstimmung (optional)
resnet_model.fit(dataset, epochs=5, validation_data=dataset)

## 7. Modell speichern und laden

In [ ]:
# Speichern
model_path = './saved_models/basic_cnn'
tf.keras.models.save_model(model, model_path)
print(f"Modell gespeichert unter: {model_path}")

# Laden
loaded_model = tf.keras.models.load_model(model_path)
loaded_model.summary()
result = loaded_model.predict(img)

## 8. Evaluation auf Evaluierungsdatensatz

In [ ]:
# Angenommen, eval_dataset ist schon definiert
results = loaded_model.evaluate(dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

## Fazit und Ausblick

- Du hast gelernt, wie man eine komplette Pipeline mit TensorFlow erstellt.
- TFRecord ermöglicht effizientes Daten-Streaming.
- Keras-API eignet sich für schnelle Prototypen und komplexe Modelle.
- **Ausblick**: Transfer Learning vertiefen, TensorFlow Lite für mobile Geräte, TensorFlow Serving für produktive Systeme.

---

# Referenzen

- https://www.tensorflow.org/guide/data
- https://www.tensorflow.org/tutorials/load_data/tfrecord
- https://www.tensorflow.org/api_docs/python/tf/keras/applications
- https://www.tensorflow.org/guide/keras